In [1]:
import torch
import torch.nn as nn
import numpy as np
import copy
import torch.nn.functional as F
import torch.optim as optim
import pickle

import torch
import torch.nn as nn
import numpy as np
import copy
import time
import torch.nn.functional as F
import torch.optim as optim
from itertools import repeat
from tqdm.notebook import tqdm
from collections import defaultdict

In [2]:
import sys
sys.path.append('./SR-GNN-master/pytorch_code')

In [3]:
from model import *
from utils import *

In [5]:
class Opt():
    def __init__(self, ):
        self.dataset = 'diginetica'
        self.batchSize = 100
        self.hiddenSize = 100
        self.epoch = 30
        self.lr = 0.001
        self.lr_dc = 0.1
        self.lr_dc_step = 3
        self.l2 = 1e-5
        self.step = 1
        self.patience = 10
        self.nonhybrid = False
        self.validation = False
        self.valid_portion = 0.1
        
opt = Opt()

In [6]:
train_data = pickle.load(open('./SR-GNN-master/datasets/' + opt.dataset + '/train.txt', 'rb'))
if opt.validation:
    train_data, valid_data = split_validation(train_data, opt.valid_portion)
    test_data = valid_data
else:
    test_data = pickle.load(open('./SR-GNN-master/datasets/' + opt.dataset + '/test.txt', 'rb'))

if opt.dataset == 'diginetica':
    n_node = 43098
elif opt.dataset == 'yoochoose1_64' or opt.dataset == 'yoochoose1_4':
    n_node = 37484
else:
    n_node = 310

In [7]:
train_seqs = pickle.load(open('./SR-GNN-master/datasets/' + opt.dataset + '/all_train_seq.txt', 'rb'))

In [22]:
set_ = set()
for i in train_data[0]:
    for j in i:
        set_.add(j)
for i in train_data[1]:
    set_.add(i)
item_num = len(set_)

item_mapping = {}
for i, item in enumerate(set_):
    item_mapping[item] = i

n = int((1-opt.valid_portion) * len(train_seqs))
d_basket_train = {k: [item_mapping[item] for item in items] for k, items in enumerate(train_seqs[:-1])}
d_basket_val = {k: [item_mapping[item] for item in items] for k, items in enumerate(train_seqs[1:])}

In [36]:
global_p = 1

In [37]:
class DataLoaderStoch():
    '''
    Класс автоматическрого создания батчей для трейна.
    '''
    def __init__(self, trans,
                 basket,
                 item_num,
                 batchsize=128,
                 max_basket_size=23,
                 shuffle=True):
        ''''
        На вход:
        trans          - список id чеков
        basket         - словарь товаров в коризне по чеку,
                         в виде списка из номеров товаров
        item_num  - кол-во товаров
        batchsize - размер батча
        shuffle   - перемешивать ли семплы
        На выход:
        батч в виде [(
                      корзины: list[list],
                      контекст: Tensor, size=(batchsize, context_dim),
                      таргетные продукты: LongTensor, size=(batchsize,),
                      клиенты: LongTensor, size=(batchsize,)
                     ),
                     таргет: Tensor, size=(batchsize,)]
        '''
        self.trans = trans
        self.basket = basket
        self.max_basket_size = max_basket_size
        self.batchsize = batchsize
        self.shuffle = shuffle
        self.prod_num = item_num
        
    def __iter__(self):  
        '''
        Метод вызывается при итерировании по объекту,
        например, через for.
        '''
        self.ids = set(range(len(self.trans)))
        return self._contaner_()
    
    def __len__(self):
        '''
        Возвращает кол-во батчей.
        '''
        batch_num = np.ceil(len(self.trans)/self.batchsize)
        return int(batch_num)
        
    def _contaner_(self):
        '''
        Метод берет подвыборку и формирует батч.
        '''
        while len(self.ids) != 0:
            if self.shuffle:
                size = min(len(self.ids), self.batchsize)
                idx_curr = np.random.choice(list(self.ids), size,
                                            replace=False)
            else:
                idx_curr = np.array(list(self.ids))[:self.batchsize]
            self.ids = self.ids.difference(idx_curr)
            yield self._make_sample_(self.trans[idx_curr])
        
    def foo(self, trans):
        '''
        Вспомогательная функция. см _make_sample_.
        '''
        trans_products = self.basket[trans]
        padding = [self.prod_num]*(self.max_basket_size - len(trans_products)+global_p)
        return (padding+trans_products)[:-1], (padding+trans_products)[-1]
        
    def _make_sample_(self, X):
        '''
        Метод возвращает готовый батч.
        '''
        self.X = X
        temp = list(map(self.foo, self.X))

        return [torch.LongTensor(np.array([row[0] for row in temp])),
                torch.LongTensor(np.array([row[1] for row in temp]))]

def generate_square_subsequent_mask(sz: int) -> torch.Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [38]:
#функция тестирования
def test(model, test_data):
    model.eval()
    hit, mrr = [], []
    batch_size = min(opt.batchSize, len(test_data[0]))
    for i in tqdm(range(0, len(test_data[0]), batch_size)):
        scores = model.predict(test_data[0][i:i+batch_size])
        sub_scores = np.array(scores.topk(20)[1])
        targets = test_data[1][i:i+batch_size]
        for score, target in zip(sub_scores, targets):
            target = model.item_mapping.get(target)
            hit.append(np.isin(target, score))
            if len(np.where(score == target)[0]) == 0:
                mrr.append(0)
            else:
                mrr.append(1 / (np.where(score == target)[0][0] + 1))
    hit = np.mean(hit) * 100
    mrr = np.mean(mrr) * 100
    return hit, mrr

#функция тестирования
def evaluate_net(net, testloader, use_cuda=True):
    net = net.eval()
    running_loss = 0.0
    # цикл по батчам внутри эпохи
    for i, data in enumerate(tqdm(testloader)):
        # берем очередной батч и его лейблы
        prods = data[0].to(device)
        labels = data[1].to(device)
        
        # получили выход сетки
        outputs = net(prods)
        
        # посчитали для этого выхода лосс
        loss = criterion(outputs, labels)
        
        #суммируемый лосс на обучении
        running_loss += float(loss)
        
    return running_loss/len(testloader)

#основная функция для обучения сети
def train_net(n_epochs, 
              net, 
              optimizer, 
              scheduler,
              criterion, 
              trainloader,
              testloader,
              test_data,
              prod_num,
              use_cuda=False,
             ):
    '''
    Функция обучения нейронной сети.
    На вход:
    n_epochs      - кол-во эпох
    net           - сеть для обучения
    optimizer     - оптимизатор для обучения
    criterion     - критерий оптимизации
    trainloader   - даталоадер для трейна
    testloader    - даталоадер для теста
    d_food_cost_idx - цены на товары
    prod_num      - кол-во продуктов
    use_cuda      - использовать ли cuda
    verbose       - если 0, то не выводит 
                    качество на валидации,
                    если > 0 выводит качество
                    на валлидации каждые verbose
                    эпох
    early_stopping_len - после какого кол-ва эпох
                    без улучшения качества 
                    надо прекратить обучение
    '''
    
    if use_cuda:
        net = net.cuda()
    
    test_loss = 0
    
    # основной цикл по всем эпохам
    for epoch in range(n_epochs):
        net = net.train()
    
        running_loss = 0.0
        # цикл по батчам внутри эпохи
        for i, data in enumerate(tqdm(trainloader)):
            # берем очередной батч и его лейблы
            prods = data[0].to(device)
            labels = data[1].to(device)

            # всегда перед вычислением градиентов зануляем их, чтобы не накапливались
            optimizer.zero_grad()

            # получили выход сетки
            outputs = net(prods)

            # посчитали для этого выхода лосс
            loss = criterion(outputs, labels)

            # вычислили градиенты loss по параметрам сети (w)
            loss.backward()

            #далем шаг по антиградиенту - обновляем веса сети
            optimizer.step()

            #суммируемый лосс на обучении
            running_loss += float(loss)
        # валидируемся
        hit, mrr = test(net, test_data)
        # test_loss = evaluate_net(net, testloader, use_cuda=use_cuda)
        # torch.save(net.state_dict(), 'net.model')
        # логируем после каждой эпохи        
        print('Epoch {}. \nTrain_loss: {:.6f}' .format(epoch + 1, running_loss / len(trainloader)))
        # print(f'Test_loss: {test_loss}')
        print(f'Test eval: hit - {hit}, mrr - {mrr}')
        print('------------------------------')
        scheduler.step()

    print('Finished Training')
    return net

In [39]:
import torch
import torch.nn as nn
import numpy as np
import copy
import torch.nn.functional as F
import torch.optim as optim
from sklearn.utils.extmath import randomized_svd


class Embedding_prod(nn.Module):
    '''
    Нейросеть для эмбеддинга товара.
    '''
    def __init__(self, data, item_num, item_mapping):
        super(Embedding_prod, self).__init__()
        self.item_mapping = item_mapping
        Dii = defaultdict(int)
        for items in tqdm(train_seqs, total=len(train_seqs)):
            session_len = len(items)
            for i in range(session_len-1):
                Dii[self.item_mapping[items[i]], self.item_mapping[items[i+1]]] += 1
        
        i = torch.LongTensor(list(Dii.keys())).T
        v = torch.LongTensor(list(Dii.values()))
        Q = torch.sparse_coo_tensor(i, v, (item_num+1, item_num+1)).to_dense()
        
        prod_embedd = nn.Embedding(item_num+1, item_num+1).requires_grad_(False)
        prod_embedd.weight = nn.Parameter(Q/(Q.sum(1)[:, None] + 1e-6))
        #prod_embedd.requires_grad = False
        self.prod_embedd = prod_embedd.requires_grad_(False)
        
    def forward(self, prods):
        prod_embedd = self.prod_embedd(prods)
        return prod_embedd


class Embedding_transformer(nn.Module):
    '''
    Нейросеть для эмбеддинга товара.
    '''
    def __init__(self, p=3):
        '''
        На вход:
        X - матрица (товары, эмбеддинг)
        use_cuda - использовать ли cuda
        
        Forward принимает на вход или
        список, содержащий номера товаров,
        или просто номер товара.
        '''
        super(Embedding_transformer, self).__init__()
        self.p = p
        self.sigm = nn.Sigmoid()
        self.base = nn.Parameter(torch.tensor(2.))
        self.params = torch.Tensor(list(range(p))[::-1])
        

    def forward(self, batch):
        t = batch[:, -self.p:, :] / (self.base**self.params[None, :, None])
        return t.sum(1)


class Net(nn.Module):
    def __init__(self, products_embedd, transformer, max_basket_size, item_mapping=item_mapping):
        '''
        На вход:
        products_embedd - объект класса Embedding_prod
        '''
        super(Net, self).__init__()
        self.embedding = products_embedd
        self.transformer = transformer
        self.max_basket_size = max_basket_size
        self.item_mapping = item_mapping
        self.prod_num = len(item_mapping)
    
    def predict(self, baskets):
        rows = []
        range_batch = range(len(baskets))
        for basket in baskets:
            basket_ = [self.item_mapping[item] for item in basket if item in item_mapping]
            padding = [self.prod_num]*(self.max_basket_size - len(basket_) + global_p - 1)
            rows.append(padding+basket_)
        return self.forward(torch.LongTensor(np.array(rows)))
    
    def forward(self, prods):
        embedd = self.embedding(prods)
        return self.transformer(embedd)

In [40]:
max_basket_size = max(map(len, train_seqs))
max_basket_size

70

In [41]:
device = 'cpu'

In [42]:
# создаем сеть
embedd_dim = 1024 # 512
prod_embedd = Embedding_prod(train_seqs, item_num, item_mapping)
transformer = Embedding_transformer(p=global_p)
net_model = Net(prod_embedd, transformer, max_basket_size, item_mapping)
net_model.to(device)

  0%|          | 0/186670 [00:00<?, ?it/s]

Net(
  (embedding): Embedding_prod(
    (prod_embedd): Embedding(43098, 43098)
  )
  (transformer): Embedding_transformer(
    (sigm): Sigmoid()
  )
)

In [43]:
net_model.embedding.prod_embedd.weight.requires_grad

False

In [44]:
net_model.state_dict()

OrderedDict([('embedding.prod_embedd.weight',
              tensor([[0.0000, 0.1429, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
                      [0.5000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
                      [0.0000, 0.0000, 0.2055,  ..., 0.0000, 0.0000, 0.0000],
                      ...,
                      [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
                      [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
                      [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])),
             ('transformer.base', tensor(2.))])

In [45]:
# net_model.load_state_dict(torch.load('net.model'))

In [46]:
learning_rate= 1e-1 # 1e-4
optimizer = optim.Adam(net_model.parameters(), lr=learning_rate, weight_decay=0)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8, verbose=True) # 0.8

# критерий оптимизации
weight = torch.ones(item_num+1)
weight[-1] = 0
criterion = nn.CrossEntropyLoss(weight=weight)

Adjusting learning rate of group 0 to 1.0000e-01.


In [47]:
torch.set_num_threads(32)

In [48]:
# создаем даталоадер для обучения
dataloader_train = DataLoaderStoch(np.array(list(d_basket_train.keys()))[:10],
                      d_basket_train,
                      item_num=item_num,
                      max_basket_size=max_basket_size,
                      batchsize=10,
                      shuffle=True)

# создаем даталоадер для валидации
dataloader_val = DataLoaderStoch(np.array(list(d_basket_val.keys())),
                      d_basket_val,
                      item_num=item_num,
                      max_basket_size=max_basket_size,
                      batchsize=1024,
                      shuffle=False)

# учим сеть
net_model = train_net(50,
                   net_model,
                   optimizer,
                   scheduler,
                   criterion,
                   dataloader_train,
                   dataloader_val,
                   test_data,
                   item_num,
                   use_cuda=False,
                  )

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/609 [00:00<?, ?it/s]

Epoch 1. 
Train_loss: 10.580825
Test eval: hit - 34.03825298235236, mrr - 12.945250365614204
------------------------------
Adjusting learning rate of group 0 to 8.0000e-02.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/609 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
net_model.state_dict()

In [ ]:
gen = next(iter(dataloader_val))

In [ ]:
net_model(gen[0]).argmax(1)[0]

In [ ]:
d_basket_val[0]

In [ ]:
next(iter(dataloader_val))[1][0]